# Chapter 12: Tools and Function Calling
**From: Zero to AI Agent**

## Overview
In this chapter, you'll learn about:
- Understanding tool use in agents
- Creating custom tools
- Built-in LangChain tools
- Function calling with LLMs
- Tool selection strategies
- Error handling in tool execution
- Building a multi-tool agent


In [ ]:
!pip install -q -r requirements.txt

from dotenv import load_dotenv
load_dotenv()

---
## Section 12.1: Understanding tool use in agents

In [ ]:
# From: no_tools_comparison.py

# From: Zero to AI Agent, Chapter 12, Section 12.1
# File: no_tools_comparison.py

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

# Load your OpenAI API key
load_dotenv()

# Create our LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Without tools - the LLM can only guess
print("WITHOUT TOOLS - The LLM tries its best:")
print("-" * 40)

response = llm.invoke([
    HumanMessage(content="What is 15,847 * 3,921?")
])

print("User: What is 15,847 * 3,921?")
print(f"AI: {response.content}")
print("\n(The AI might attempt the math, but could be wrong!)")

# Try another question that needs real-world data
response2 = llm.invoke([
    HumanMessage(content="What's the current temperature in Tokyo?")
])

print("\nUser: What's the current temperature in Tokyo?")
print(f"AI: {response2.content}")
print("\n(The AI can only guess or use outdated training data!)")


In [ ]:
# From: first_tool_agent.py

# From: Zero to AI Agent, Chapter 12, Section 12.1
# File: first_tool_agent.py

from langchain_openai import ChatOpenAI
from langchain_classic.agents import create_react_agent, AgentExecutor
from langchain_community.tools import DuckDuckGoSearchRun
from langsmith import Client
from dotenv import load_dotenv
import os

load_dotenv()

# Create our LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Use a pre-built tool (we'll learn to make our own in 12.2!)
search = DuckDuckGoSearchRun()

# Create an agent with the search tool
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=langsmith_api_key)
prompt = client.pull_prompt("hwchase17/react")
agent = create_react_agent(llm, [search], prompt)
executor = AgentExecutor(
    agent=agent, 
    tools=[search], 
    verbose=True,  # Show the thinking process!
    max_iterations=3
)

# Ask something that needs current information
print("AGENT WITH SEARCH CAPABILITY")
print("=" * 50)

result = executor.invoke({
    "input": "What is Python programming language known for?"
})

print(f"\nFinal Answer: {result['output']}")

---
### Section 12.1 Exercises

### Exercise 12.1.1: Tool or No Tool?

For each scenario below, decide whether you need a tool or if pure LLM is sufficient. Explain why!

1. Write a haiku about summer
2. Get today's date
3. Explain quantum physics
4. Check if a file exists
5. Translate 'hello' to Spanish
6. Find the latest news about AI
7. Generate a business name
8. Calculate compound interest

For each one, answer:
- **Tool or No Tool?**
- **Why?** What makes you choose one over the other?
- **If tool needed:** What kind of tool would you use?
- **If no tool:** What would happen if you tried to use a tool anyway?

Remember the golden rule: Current data and precise computation need tools; creativity and knowledge don't!

In [ ]:
# Your code here


### Exercise 12.1.2: Trace the Tool Flow

Given this conversation, trace what happens at each step behind the scenes:
```
User: "Search for information about LangChain and calculate 15% of 2000"

AI: "I'll help you with both tasks.

LangChain is a framework for developing applications powered by 
language models. It was created by Harrison Chase and provides 
tools for building context-aware reasoning applications.

15% of 2000 is 300."
```

Write out each step that happened:
- Step 1: User sends message with two requests
- Step 2: AI recognizes... [continue]
- Step 3: ...
- Step 4: ...

Think about:
- How did the AI know to use two different tools?
- Which tool was called first? Why?
- How did the AI combine the results into one response?

In [ ]:
# Your code here


### Exercise 12.1.3: Design a Tool Set

You're building an AI assistant for a small business (a local bakery). Design 5 tools it would need:

For each tool, describe:
- **Tool name:** What you'd call it
- **Purpose:** What it does
- **When used:** Example user requests that would trigger it
- **Inputs:** What information it needs
- **Outputs:** What it returns
- **Safety considerations:** What could go wrong and how to prevent it

Example:
- **Tool name:** `check_inventory`
- **Purpose:** Check current stock levels of ingredients
- **When used:** "Do we have enough flour for tomorrow's orders?"
- **Inputs:** Ingredient name, optional date
- **Outputs:** Current quantity and unit (e.g., "150 pounds of flour")
- **Safety:** Read-only database access, validate ingredient names against known list

Now design 5 tools that would make this bakery assistant truly helpful! Think about daily operations, customer service, and business management.

In [ ]:
# Your code here


---
## Section 12.2: Creating custom tools

In [ ]:
# From: first_custom_tool.py

# From: Zero to AI Agent, Chapter 12, Section 12.2
# File: first_custom_tool.py

from langchain_core.tools import Tool

# Step 1: Create a regular Python function
def greet(name: str) -> str:
    """Generate a friendly greeting."""
    return f"Hello {name}! Welcome to the world of custom tools!"

# Step 2: Wrap it as a LangChain tool
greeting_tool = Tool(
    name="Greeter",  # What the LLM will call this
    func=greet,      # The function to run
    description="Use this to greet someone by name"  # When to use it
)

# Test it directly
result = greeting_tool.func("Alice")
print(result)  # "Hello Alice! Welcome to the world of custom tools!"


In [ ]:
# From: calculator_tool.py

# From: Zero to AI Agent, Chapter 12, Section 12.2
# File: calculator_tool.py

from langchain_core.tools import Tool
import re

def safe_calculator(expression: str) -> str:
    """
    A calculator that safely evaluates mathematical expressions.
    Returns the result or an error message.
    """
    try:
        # Clean the expression - remove spaces and commas
        expression = expression.replace(" ", "").replace(",", "")
        
        # Only allow numbers and basic math operators
        if not re.match(r'^[0-9+\-*/().\s]+$', expression):
            return "Error: Only numbers and +, -, *, /, () allowed"
        
        # Calculate the result
        result = eval(expression)
        
        # Format nicely for large numbers
        if isinstance(result, (int, float)):
            if result > 1000:
                return f"{result:,.2f}"
            return str(result)
            
    except ZeroDivisionError:
        return "Error: Cannot divide by zero"
    except Exception as e:
        return f"Error: Invalid expression"

# Create the tool
calculator = Tool(
    name="Calculator",
    func=safe_calculator,
    description="Performs mathematical calculations. Input should be a math expression like '2+2' or '100*3.14'"
)

# Test it
print(calculator.func("2 + 2"))           # "4"
print(calculator.func("1000 * 1.5"))      # "1,500.00"
print(calculator.func("10 / 0"))          # "Error: Cannot divide by zero"
print(calculator.func("hack()"))          # "Error: Only numbers and +, -, *, /, () allowed"


In [ ]:
# From: tool_io_examples.py

# From: Zero to AI Agent, Chapter 12, Section 12.2
# File: tool_io_examples.py

from langchain_core.tools import Tool
import json
from datetime import datetime

# Tool with multiple inputs (passed as formatted string)
def create_reminder(reminder_text: str) -> str:
    """
    Create a reminder with date and message.
    Format: 'DATE|MESSAGE' like '2024-12-25|Christmas Day'
    """
    try:
        parts = reminder_text.split('|')
        if len(parts) != 2:
            return "Error: Use format 'DATE|MESSAGE'"
        
        date_str, message = parts
        # Validate date
        reminder_date = datetime.strptime(date_str.strip(), '%Y-%m-%d')
        
        return f"✓ Reminder set for {date_str}: {message}"
    except ValueError:
        return "Error: Invalid date format. Use YYYY-MM-DD"

# Tool that returns structured data (as formatted string)
def get_weather(city: str) -> str:
    """Get weather for a city (simulated)."""
    # In real life, this would call an API
    weather_data = {
        "New York": {"temp": 72, "condition": "Sunny", "humidity": 65},
        "London": {"temp": 59, "condition": "Cloudy", "humidity": 80},
        "Tokyo": {"temp": 68, "condition": "Clear", "humidity": 55}
    }
    
    if city in weather_data:
        data = weather_data[city]
        return f"Weather in {city}: {data['temp']}°F, {data['condition']}, {data['humidity']}% humidity"
    else:
        return f"Weather data not available for {city}"

# Create the tools
reminder_tool = Tool(
    name="ReminderCreator",
    func=create_reminder,
    description="Create a reminder. Input format: 'YYYY-MM-DD|message text'"
)

weather_tool = Tool(
    name="WeatherChecker", 
    func=get_weather,
    description="Get current weather for a city. Input: city name"
)

# Test them
print(reminder_tool.func("2024-12-25|Christmas Day"))
print(weather_tool.func("New York"))


In [ ]:
# From: description_importance.py

# From: Zero to AI Agent, Chapter 12, Section 12.2
# File: description_importance.py

from langchain_core.tools import Tool

def same_function(text: str) -> str:
    """This function does text analysis."""
    words = text.split()
    return f"Word count: {len(words)}"

# Same function, different descriptions
vague_tool = Tool(
    name="Analyzer",
    func=same_function,
    description="Analyzes text"  # Too vague!
)

clear_tool = Tool(
    name="WordCounter",
    func=same_function,
    description="Counts the number of words in text. Use when someone asks for word count, length, or size of text."
)

# The LLM will reliably use clear_tool for "How many words are in this?"
# But might not use vague_tool for the same question!


In [ ]:
# From: bulletproof_tool.py

# From: Zero to AI Agent, Chapter 12, Section 12.2
# File: bulletproof_tool.py

from langchain_core.tools import Tool
import re
from typing import Optional
import time

def bulletproof_search(query: str) -> str:
    """
    A production-ready search tool with all safety features.
    """
    # 1. Input validation
    if not query or not isinstance(query, str):
        return "Error: Please provide a search query"
    
    # 2. Length limits
    if len(query) > 200:
        return "Error: Query too long (max 200 characters)"
    
    # 3. Clean dangerous characters
    query = re.sub(r'[^\w\s\-.]', '', query)
    query = query.strip()
    
    if not query:
        return "Error: Invalid search query"
    
    # 4. Rate limiting (in production, use proper rate limiting)
    time.sleep(0.1)  # Prevent hammering
    
    try:
        # 5. Timeout for external calls
        # In real tool: requests.get(url, timeout=5)
        
        # 6. Simulated search results
        results = f"Search results for '{query}': Found 3 relevant articles..."
        
        # 7. Output sanitization
        return results[:500]  # Limit response size
        
    except Exception as e:
        # 8. Generic error handling
        return "Error: Search temporarily unavailable"

# Create production-ready tool
search_tool = Tool(
    name="WebSearch",
    func=bulletproof_search,
    description="Search the web for information. Input: search query (max 200 chars)"
)

# Test edge cases
print(search_tool.func(""))                    # Error handling
print(search_tool.func("Normal search"))       # Success
print(search_tool.func("x" * 300))            # Length limit
print(search_tool.func("'; DROP TABLE;"))      # SQL injection attempt - cleaned


In [ ]:
# From: api_tool_pattern.py

# From: Zero to AI Agent, Chapter 12, Section 12.2
# File: api_tool_pattern.py

from langchain_core.tools import Tool
import os
from typing import Optional

def create_api_tool(endpoint_name: str):
    """
    Factory function to create API tools.
    This is a pattern - adapt for your specific APIs!
    """
    def api_caller(parameters: str) -> str:
        # Check for API key
        api_key = os.getenv(f"{endpoint_name.upper()}_API_KEY")
        if not api_key:
            return f"Error: {endpoint_name} API key not configured"
        
        try:
            # In real implementation:
            # import requests
            # response = requests.get(
            #     f"https://api.{endpoint_name}.com/v1/query",
            #     params={"q": parameters},
            #     headers={"Authorization": f"Bearer {api_key}"},
            #     timeout=10
            # )
            # return response.json()
            
            # Simulated response
            return f"API response for {parameters} from {endpoint_name}"
            
        except TimeoutError:
            return f"Error: {endpoint_name} API timeout"
        except Exception as e:
            return f"Error: {endpoint_name} API failed"
    
    return Tool(
        name=f"{endpoint_name.title()}API",
        func=api_caller,
        description=f"Query the {endpoint_name} API. Input: query parameters"
    )

# Create tools for different APIs
weather_api = create_api_tool("weather")
news_api = create_api_tool("news")
stock_api = create_api_tool("stocks")

# Each tool follows the same pattern but connects to different services


In [ ]:
# From: tool_composition.py

# From: Zero to AI Agent, Chapter 12, Section 12.2
# File: tool_composition.py

from langchain_core.tools import Tool
import json

# Tool 1: Fetches data
def fetch_data(source: str) -> str:
    """Fetch data from a source."""
    # Simulated data fetching
    data = {
        "sales": [100, 150, 120, 180, 200],
        "costs": [80, 90, 85, 95, 100]
    }
    return json.dumps(data)

# Tool 2: Analyzes data
def analyze_data(json_data: str) -> str:
    """Analyze data and return insights."""
    try:
        data = json.loads(json_data)
        sales = data.get("sales", [])
        costs = data.get("costs", [])
        
        total_sales = sum(sales)
        total_costs = sum(costs)
        profit = total_sales - total_costs
        margin = (profit / total_sales * 100) if total_sales > 0 else 0
        
        return f"Analysis: Total Sales: ${total_sales}, Total Costs: ${total_costs}, Profit: ${profit}, Margin: {margin:.1f}%"
    except:
        return "Error: Invalid data format for analysis"

# Tool 3: Formats reports
def format_report(analysis: str) -> str:
    """Format analysis into a nice report."""
    if "Error" in analysis:
        return analysis
    
    report = f"""
    📊 BUSINESS REPORT
    ==================
    {analysis}
    
    Status: ✅ Profitable
    Recommendation: Continue current strategy
    """
    return report.strip()

# Create the tool chain
fetch_tool = Tool(name="DataFetcher", func=fetch_data, 
                  description="Fetch business data from a source")
analyze_tool = Tool(name="DataAnalyzer", func=analyze_data,
                    description="Analyze JSON data and return insights")
format_tool = Tool(name="ReportFormatter", func=format_report,
                   description="Format analysis into a professional report")

# These tools naturally work together:
# 1. Fetch data → 2. Analyze it → 3. Format report
# The LLM orchestrates this flow automatically!


---
### Section 12.2 Exercises

### Exercise 12.2.1: Unit Converter Tool (Easy)

Create a tool that converts between units (meters to feet, kg to pounds, etc.):
- Handle at least 3 types of conversions
- Parse input like "5 meters to feet"
- Return clear, formatted results
- Handle invalid conversions gracefully

In [ ]:
# Your code here


### Exercise 12.2.2: Text Processor Tool Suite (Medium)

Build three related tools that work together:
1. `text_stats`: Returns word count, sentence count, average word length
2. `extract_keywords`: Finds the 5 most common meaningful words
3. `summarize`: Creates a one-sentence summary

Make sure the outputs of one can be inputs to another!

In [ ]:
# Your code here


### Exercise 12.2.3: Smart File Manager (Hard)

Create a tool that manages a simple file system:
- Commands: "create file.txt with [content]", "read file.txt", "list files", "delete file.txt"
- Store files in a dictionary (simulate a file system)
- Add safety: prevent overwriting without confirmation
- Return user-friendly messages
- Bonus: Add a search function to find files containing specific text

In [ ]:
# Your code here


---
## Section 12.3: Built-in LangChain tools

In [ ]:
# From: exploring_duckduckgo.py

# From: Zero to AI Agent, Chapter 12, Section 12.3
# File: exploring_duckduckgo.py

from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the search tool - that's it!
search_tool = DuckDuckGoSearchRun()

# Test it directly - just like calling a function!
print("Testing DuckDuckGo Search Tool")
print("=" * 50)

# Search for something current
result = search_tool.run("Python programming language latest version 2024")
print("Search for Python version:")
print(result[:300] + "...")  # First 300 chars
print()

# Search for news
news_result = search_tool.run("artificial intelligence breakthrough today")
print("Search for AI news:")
print(news_result[:300] + "...")
print()

# Search for factual information
fact_result = search_tool.run("height of Mount Everest in meters")
print("Search for facts:")
print(fact_result[:300] + "...")


In [ ]:
# From: exploring_wikipedia.py

# From: Zero to AI Agent, Chapter 12, Section 12.3
# File: exploring_wikipedia.py

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Configure Wikipedia tool
wikipedia_tool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(
        top_k_results=1,  # Number of results to return
        doc_content_chars_max=1000  # Max characters per result
    )
)

print("Testing Wikipedia Tool")
print("=" * 50)

# Look up a programming language
result = wikipedia_tool.run("Python (programming language)")
print("Python programming language:")
print(result[:400] + "...")
print()

# Look up a concept
ml_result = wikipedia_tool.run("Machine Learning")
print("Machine Learning:")
# Get just the first two sentences
sentences = ml_result.split('. ')[:2]
print('. '.join(sentences) + '.')
print()

# Look up a person
turing_result = wikipedia_tool.run("Alan Turing")
print("Alan Turing:")
sentences = turing_result.split('. ')[:2]
print('. '.join(sentences) + '.')


In [ ]:
# From: exploring_file_tools.py

# From: Zero to AI Agent, Chapter 12, Section 12.3
# File: exploring_file_tools.py

from langchain_community.tools import (
    WriteFileTool,
    ReadFileTool,
    ListDirectoryTool
)
import tempfile
import os

# Create a temporary directory to work in safely
working_dir = tempfile.mkdtemp(prefix="tools_exploration_")
print(f"Working directory: {working_dir}")
print("=" * 50)

# Initialize file tools with the working directory
write_tool = WriteFileTool(root_dir=working_dir)
read_tool = ReadFileTool(root_dir=working_dir)
list_tool = ListDirectoryTool(root_dir=working_dir)

# Test 1: Write a file
print("\n1. Testing WriteFileTool:")
write_result = write_tool.run({
    "file_path": "test_note.txt",
    "text": "Hello from LangChain tools!\nThis is a test file."
})
print(f"   Result: {write_result}")

# Test 2: List directory contents
print("\n2. Testing ListDirectoryTool:")
list_result = list_tool.run({"dir_path": "."})
print(f"   Contents: {list_result}")

# Test 3: Read the file back
print("\n3. Testing ReadFileTool:")
read_result = read_tool.run({"file_path": "test_note.txt"})
print(f"   File contents: {read_result}")

# Test 4: Create a more complex file
print("\n4. Creating a Python script:")
python_code = '''def greet(name):
    return f"Hello, {name}!"

if __name__ == "__main__":
    print(greet("World"))
'''
write_tool.run({
    "file_path": "hello.py",
    "text": python_code
})
print("   Created hello.py")

# List files again
final_list = list_tool.run({"dir_path": "."})
print(f"\n5. Final directory contents: {final_list}")

# Clean up
print(f"\nFiles created in: {working_dir}")
print("(This is a temporary directory)")


In [ ]:
# From: exploring_python_repl.py

# From: Zero to AI Agent, Chapter 12, Section 12.3
# File: exploring_python_repl.py

from langchain_experimental.tools import PythonREPLTool

# Create the Python REPL tool
python_tool = PythonREPLTool()

print("Testing Python REPL Tool")
print("=" * 50)
print("⚠️  This tool can execute ANY Python code - use with caution!")
print()

# Test 1: Simple calculation
print("1. Simple math:")
result = python_tool.run("print(2 ** 10)")
print(f"   2^10 = {result}")
print()

# Test 2: Generate Fibonacci sequence
print("2. Fibonacci sequence:")
fibonacci_code = """
def fibonacci(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]
    else:
        fib = [0, 1]
        for i in range(2, n):
            fib.append(fib[-1] + fib[-2])
        return fib

result = fibonacci(10)
print(f"First 10 Fibonacci numbers: {result}")
print(f"Sum: {sum(result)}")
"""
result = python_tool.run(fibonacci_code)
print(f"   {result}")
print()

# Test 3: Data analysis
print("3. Data analysis:")
analysis_code = """
data = [23, 45, 67, 89, 12, 34, 56, 78, 90, 11]
mean = sum(data) / len(data)
maximum = max(data)
minimum = min(data)
print(f"Data: {data}")
print(f"Mean: {mean:.2f}, Max: {maximum}, Min: {minimum}")
"""
result = python_tool.run(analysis_code)
print(f"   {result}")


In [ ]:
# From: tools_working_together.py

# From: Zero to AI Agent, Chapter 12, Section 12.3
# File: tools_working_together.py

from langchain_community.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WriteFileTool
import tempfile

# Initialize our tools
search_tool = DuckDuckGoSearchRun()
wikipedia_tool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
)
working_dir = tempfile.mkdtemp(prefix="research_")
write_tool = WriteFileTool(root_dir=working_dir)

print("Demonstrating How Tools Complement Each Other")
print("=" * 50)

# Research task: Learn about a topic from multiple angles
topic = "Large Language Models"

# Step 1: Get established facts from Wikipedia
print(f"\n1. Wikipedia (Established Facts) about {topic}:")
wiki_result = wikipedia_tool.run(topic)
wiki_summary = wiki_result[:200] + "..."
print(wiki_summary)

# Step 2: Get current developments from search
print(f"\n2. Web Search (Recent News) about {topic}:")
search_query = f"{topic} breakthrough 2024 news"
search_result = search_tool.run(search_query)
search_summary = search_result[:200] + "..."
print(search_summary)

# Step 3: Save the combined research
print(f"\n3. Saving Research to File:")
research_report = f"""Research Report: {topic}
Generated on: {tempfile.gettempdir()}

ESTABLISHED FACTS (Wikipedia):
{wiki_result[:400]}

RECENT DEVELOPMENTS (Web Search):
{search_result[:400]}

Summary: This report combines foundational knowledge with current developments.
"""

write_tool.run({
    "file_path": "research_report.txt",
    "text": research_report
})
print(f"   Report saved to: {working_dir}/research_report.txt")

print("\n" + "=" * 50)
print("Notice how each tool serves a different purpose:")
print("- Wikipedia: Authoritative, stable information")
print("- Web Search: Current events and recent developments")
print("- File Tools: Persistence and report generation")
print("\nWhen agents use these together, they become research assistants!")


---
### Section 12.3 Exercises

### Exercise 12.3.1: Tool Explorer (Easy)

Test each tool type and document what it returns:
- Use DuckDuckGo to search for 3 different types of queries (news, facts, tutorials)
- Use Wikipedia to look up 3 topics (person, place, concept)
- Compare the type and quality of information each provides
- Which tool would be better for which type of question?

In [ ]:
# Your code here


### Exercise 12.3.2: Research Workflow (Medium)

Create a research workflow using tools directly (not with an agent):
- Pick a technology topic (like "quantum computing" or "blockchain")
- Use Wikipedia to get foundational information
- Use DuckDuckGo to find recent developments
- Use file tools to create a structured report
- Save the report with clear sections for "Background" and "Recent News"

In [ ]:
# Your code here


### Exercise 12.3.3: Data Processing Pipeline (Hard)

Build a data processing workflow:
- Use the Python REPL tool to generate sample data (like 100 random numbers)
- Use the Python REPL to calculate statistics (mean, median, mode, std deviation)
- Create a formatted report of the analysis
- Use file tools to save both the raw data and the analysis report
- Test edge cases: What happens with empty data? Invalid calculations?


Remember: We're exploring these tools directly for now. In the next section, you'll learn to connect them to LLMs!

In [ ]:
# Your code here


---
## Section 12.4: Function calling with LLMs

In [ ]:
# From: function_calling_intro.py

# From: Zero to AI Agent, Chapter 12, Section 12.4
# File: function_calling_intro.py

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.tools import Tool
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json

load_dotenv()

# Define the input schema
class WeatherInput(BaseModel):
    location: str = Field(description="The city or location to get weather for")
    unit: str = Field(default="fahrenheit", description="Temperature unit: 'fahrenheit' or 'celsius'")

# Define a simple function
def get_weather(location: str, unit: str = "fahrenheit") -> str:
    """Get the weather for a location."""
    # Simulated weather data
    weather_data = {
        "New York": {"f": 72, "c": 22},
        "London": {"f": 59, "c": 15},
        "Tokyo": {"f": 68, "c": 20}
    }
    
    if location in weather_data:
        temp = weather_data[location]["c" if unit == "celsius" else "f"]
        unit_symbol = "°C" if unit == "celsius" else "°F"
        return f"The weather in {location} is {temp}{unit_symbol}"
    return f"Weather data not available for {location}"

# Convert to a tool with schema
weather_tool = Tool(
    name="get_weather",
    func=get_weather,
    description="Get weather for a location",
    args_schema=WeatherInput  # THIS IS CRITICAL!
)

# Modern way: Bind tools directly to the model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools([weather_tool])

# Ask a question that needs the tool
response = llm_with_tools.invoke([
    HumanMessage(content="What's the weather in New York?")
])

print("LLM Response:", response)

# If the LLM wants to use a tool, it tells us EXACTLY how
if response.tool_calls:
    tool_call = response.tool_calls[0]
    print(f"\nTool to call: {tool_call['name']}")
    print(f"Arguments: {tool_call['args']}")
    
    # Execute the function with the exact arguments
    result = get_weather(**tool_call['args'])
    print(f"Result: {result}")

In [ ]:
# From: structured_inputs.py

# From: Zero to AI Agent, Chapter 12, Section 12.4
# File: structured_inputs.py

from typing import Optional
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.tools import StructuredTool
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

# Define the structure of your inputs using Pydantic
class FlightSearchInput(BaseModel):
    origin: str = Field(description="Departure city")
    destination: str = Field(description="Arrival city")  
    date: str = Field(description="Date in YYYY-MM-DD format")
    passengers: int = Field(default=1, description="Number of passengers")
    class_type: Optional[str] = Field(default="economy", description="economy, business, or first")

def search_flights(
    origin: str,
    destination: str,
    date: str,
    passengers: int = 1,
    class_type: str = "economy"
) -> str:
    """Search for flights based on criteria."""
    # Simulated search
    price = 200 * passengers
    if class_type == "business":
        price *= 3
    elif class_type == "first":
        price *= 5
    
    return f"Found flights from {origin} to {destination} on {date}: ${price} for {passengers} passenger(s) in {class_type}"

# Create a structured tool
flight_tool = StructuredTool.from_function(
    func=search_flights,
    name="search_flights",
    description="Search for available flights",
    args_schema=FlightSearchInput
)

# Use with function calling
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools([flight_tool])

# Complex request
response = llm_with_tools.invoke([
    HumanMessage(content="Find me business class flights from New York to London on March 15th, 2024 for 2 people")
])

if response.tool_calls:
    tool_call = response.tool_calls[0]
    print("Structured input parsed perfectly:")
    for key, value in tool_call['args'].items():
        print(f"  {key}: {value}")
    
    # Execute with confidence - parameters are guaranteed correct!
    result = search_flights(**tool_call['args'])
    print(f"\nResult: {result}")


In [ ]:
# From: reliable_selection.py

# From: Zero to AI Agent, Chapter 12, Section 12.4
# File: reliable_selection.py

from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

# Define multiple tools with clear purposes
def calculate(expression: str) -> str:
    """Perform mathematical calculations."""
    try:
        return str(eval(expression))
    except:
        return "Calculation error"

def translate(text: str, target_language: str) -> str:
    """Translate text to another language."""
    # Simulated translation
    translations = {
        "spanish": {"hello": "hola", "goodbye": "adiós"},
        "french": {"hello": "bonjour", "goodbye": "au revoir"}
    }
    # Simple word lookup (real implementation would use an API)
    return f"Translation to {target_language}: {text} → ..."

def search(query: str) -> str:
    """Search for current information."""
    return f"Search results for '{query}': Latest news and updates..."

# Create tools
tools = [
    Tool(name="calculator", func=calculate, description="For math calculations"),
    Tool(name="translator", func=translate, description="For language translation"),
    Tool(name="search", func=search, description="For current information")
]

# Bind all tools
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# Test different queries
test_queries = [
    "What is 25 * 4?",
    "How do you say hello in Spanish?",
    "What's the latest news about AI?"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    response = llm_with_tools.invoke([HumanMessage(content=query)])
    
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        print(f"Selected tool: {tool_call['name']}")
        print(f"Confidence: High (structured selection)")
    else:
        print("No tool needed")


In [ ]:
# From: tool_response_flow.py

# From: Zero to AI Agent, Chapter 12, Section 12.4
# File: tool_response_flow.py

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import Tool
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json

load_dotenv()

# Define input schema for stock price tool
class StockInput(BaseModel):
    symbol: str = Field(description="Stock ticker symbol (e.g., AAPL, GOOGL, MSFT)")

def get_stock_price(symbol: str) -> str:
    """Get current stock price."""
    # Simulated prices
    prices = {
        "AAPL": 178.50,
        "GOOGL": 142.30,
        "MSFT": 405.20
    }
    return json.dumps({
        "symbol": symbol,
        "price": prices.get(symbol.upper(), 0),
        "currency": "USD"
    })

# Create tool with proper args_schema
stock_tool = Tool(
    name="get_stock_price",
    func=get_stock_price,
    description="Get current stock price for a symbol",
    args_schema=StockInput  # Required for bind_tools!
)

# Initialize
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools([stock_tool])

# Start conversation
messages = [
    HumanMessage(content="What's the current price of Apple stock?")
]

# Step 1: LLM decides to use a tool
ai_response = llm_with_tools.invoke(messages)
messages.append(ai_response)

print("Step 1 - LLM wants to use a tool:")
if ai_response.tool_calls:
    print(f"  Tool: {ai_response.tool_calls[0]['name']}")
    print(f"  Args: {ai_response.tool_calls[0]['args']}")
    
    # Step 2: Execute the tool
    tool_call = ai_response.tool_calls[0]
    tool_result = get_stock_price(**tool_call['args'])
    
    # Step 3: Send tool result back to LLM
    tool_message = ToolMessage(
        content=tool_result,
        tool_call_id=tool_call['id']
    )
    messages.append(tool_message)
    
    print("\nStep 2 - Tool execution result:")
    print(f"  {tool_result}")
    
    # Step 4: LLM incorporates result into final response
    final_response = llm_with_tools.invoke(messages)
    
    print("\nStep 3 - Final response to user:")
    print(f"  {final_response.content}")
else:
    print("No tool calls were made")

In [ ]:
# From: parallel_tools.py

# From: Zero to AI Agent, Chapter 12, Section 12.4
# File: parallel_tools.py

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.tools import Tool
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

# Define input schema for city-based tools
class CityInput(BaseModel):
    city: str = Field(description="Name of the city")

# Define multiple tools
def get_temperature(city: str) -> str:
    temps = {"New York": 72, "London": 59, "Tokyo": 68}
    return f"{temps.get(city, 'Unknown')}°F"

def get_time(city: str) -> str:
    times = {"New York": "10:30 AM", "London": "3:30 PM", "Tokyo": "11:30 PM"}
    return times.get(city, "Unknown")

def get_population(city: str) -> str:
    pops = {"New York": "8.3M", "London": "9.5M", "Tokyo": "14M"}
    return pops.get(city, "Unknown")

# Create tools with proper args_schema
tools = [
    Tool(
        name="get_temperature",
        func=get_temperature,
        description="Get temperature for a city",
        args_schema=CityInput  # Required for bind_tools!
    ),
    Tool(
        name="get_time",
        func=get_time,
        description="Get current time for a city",
        args_schema=CityInput  # Required for bind_tools!
    ),
    Tool(
        name="get_population",
        func=get_population,
        description="Get population for a city",
        args_schema=CityInput  # Required for bind_tools!
    )
]

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# Ask for multiple pieces of information
response = llm_with_tools.invoke([
    HumanMessage(content="Tell me the temperature, time, and population of New York")
])

print("Parallel tool calls:")
if response.tool_calls:
    for i, tool_call in enumerate(response.tool_calls, 1):
        print(f"\nTool Call {i}:")
        print(f"  Function: {tool_call['name']}")
        print(f"  Arguments: {tool_call['args']}")
        
        # Execute the function properly
        if tool_call['name'] == 'get_temperature':
            result = get_temperature(**tool_call['args'])
        elif tool_call['name'] == 'get_time':
            result = get_time(**tool_call['args'])
        elif tool_call['name'] == 'get_population':
            result = get_population(**tool_call['args'])
        else:
            result = "Unknown tool"
        
        print(f"  Result: {result}")

print("\nAll three tools called in parallel - much faster than sequential!")

In [ ]:
# From: function_error_handling.py

# From: Zero to AI Agent, Chapter 12, Section 12.4
# File: function_error_handling.py

from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage, ToolMessage
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json

load_dotenv()

# Define input schema for the function
class DivideInput(BaseModel):
    a: float = Field(description="The dividend (number to be divided)")
    b: float = Field(description="The divisor (number to divide by)")

def divide_numbers(a: float, b: float) -> str:
    """Safely divide two numbers with error handling."""
    if b == 0:
        # Return structured error that LLM can understand
        return json.dumps({
            "success": False,
            "error": "Division by zero",
            "suggestion": "Please provide a non-zero divisor"
        })
    
    result = a / b
    return json.dumps({
        "success": True,
        "result": result,
        "calculation": f"{a} ÷ {b} = {result}"
    })

# Create tool with proper args_schema
calc_tool = Tool(
    name="divide_numbers",
    func=divide_numbers,
    description="Divide two numbers safely with error handling",
    args_schema=DivideInput  # THIS IS REQUIRED FOR bind_tools!
)

# Bind tool to LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tool = llm.bind_tools([calc_tool])

# Test cases including error scenarios
test_cases = [
    "What is 100 divided by 5?",
    "Divide 50 by 0",  # This will trigger error handling
    "Calculate 15.5 divided by 2.5"
]

for query in test_cases:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print('-'*60)
    
    # Get LLM response
    response = llm_with_tool.invoke([HumanMessage(content=query)])
    
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        print(f"Tool called: {tool_call['name']}")
        print(f"Arguments: {tool_call['args']}")
        
        # Execute the tool
        result = divide_numbers(**tool_call['args'])
        print(f"Tool result: {result}")
        
        # Parse result to check for errors
        result_data = json.loads(result)
        
        # Send result back to LLM for final response
        tool_message = ToolMessage(
            content=result,
            tool_call_id=tool_call['id']
        )
        
        final_response = llm_with_tool.invoke([
            HumanMessage(content=query),
            response,
            tool_message
        ])
        
        print(f"\nFinal answer: {final_response.content}")
        
        # Show how the error was handled
        if not result_data["success"]:
            print(f"⚠️ Error handled gracefully: {result_data['error']}")
            print(f"💡 Suggestion: {result_data['suggestion']}")
    else:
        print("No tool was called")

---
### Section 12.4 Exercises

### Exercise 12.4.1: Weather Assistant with Units (Easy)

Create a weather tool that:
- Accepts city and temperature unit (Celsius/Fahrenheit)
- Uses function calling for structured inputs
- Returns formatted weather data
- Test with: "What's the weather in London in Celsius?"

In [ ]:
# Your code here


### Exercise 12.4.2: Multi-Step Calculator (Medium)

Build a calculator that can:
- Handle multiple operations in one request
- Use parallel function calls for independent calculations
- Return all results together
- Test with: "Calculate 15\*4, 100/5, and 78+22"

In [ ]:
# Your code here


### Exercise 12.4.3: Smart Task Manager (Hard)

Create a task management system with function calling:
- Add tasks with title, priority, and due date
- List tasks (all, by priority, by date)
- Mark tasks complete
- Use structured inputs for all operations
- Handle errors gracefully (invalid dates, missing tasks)
- Test with complex requests like "Add a high priority task to call mom tomorrow"

In [ ]:
# Your code here


---
## Section 12.5: Tool selection strategies

In [ ]:
# From: tool_selection_basics.py

# From: Zero to AI Agent, Chapter 12, Section 12.5
# File: tool_selection_basics.py

from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

# Create tools with varying specificity
def general_search(query: str) -> str:
    return f"General web results for: {query}"

def news_search(query: str) -> str:
    return f"Latest news about: {query}"

def academic_search(query: str) -> str:
    return f"Academic papers about: {query}"

def weather_check(location: str) -> str:
    return f"Weather in {location}: Sunny, 72°F"

# Create tools with clear, specific descriptions
tools = [
    Tool(
        name="general_search",
        func=general_search,
        description="Search the web for any kind of information"
    ),
    Tool(
        name="news_search",
        func=news_search,
        description="Search specifically for recent news and current events"
    ),
    Tool(
        name="academic_search",
        func=academic_search,
        description="Search for academic papers, research, and scholarly articles"
    ),
    Tool(
        name="weather",
        func=weather_check,
        description="Get current weather for a specific location"
    )
]

# Create LLM with tools
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# Test different queries to see tool selection
test_queries = [
    "What's the weather in Paris?",
    "Find recent news about AI",
    "Search for information about Python",
    "Find academic research on machine learning"
]

print("TOOL SELECTION DEMONSTRATION")
print("=" * 50)

for query in test_queries:
    response = llm_with_tools.invoke([HumanMessage(content=query)])
    
    print(f"\nQuery: '{query}'")
    if response.tool_calls:
        selected_tool = response.tool_calls[0]['name']
        print(f"Selected: {selected_tool}")
        print(f"Reasoning: Matched keywords and intent to tool description")
    else:
        print("No tool selected")


In [ ]:
# From: description_improvement.py

# From: Zero to AI Agent, Chapter 12, Section 12.5
# File: description_improvement.py

from langchain_core.tools import Tool

# Bad descriptions - vague and ambiguous
bad_calculator = Tool(
    name="calc",
    func=lambda x: eval(x),
    description="calculator"  # Too vague!
)

bad_search = Tool(
    name="srch",
    func=lambda x: f"Results for {x}",
    description="searches stuff"  # What stuff? When?
)

# Good descriptions - clear and specific
good_calculator = Tool(
    name="calculator",
    func=lambda x: eval(x),
    description=(
        "Use this for mathematical calculations and arithmetic. "
        "Input: mathematical expression like '2+2' or '15*3.14'. "
        "Output: numerical result as a string."
    )
)

good_search = Tool(
    name="web_search",
    func=lambda x: f"Results for {x}",
    description=(
        "Use this to search the web for current information, news, or facts. "
        "Best for: recent events, current data, or topics after 2021. "
        "Input: search query string. "
        "Output: relevant web snippets and summaries."
    )
)

print("DESCRIPTION COMPARISON")
print("=" * 50)

print("\n❌ BAD DESCRIPTIONS:")
print(f"Calculator: '{bad_calculator.description}'")
print(f"Search: '{bad_search.description}'")

print("\n✅ GOOD DESCRIPTIONS:")
print(f"Calculator: '{good_calculator.description}'")
print(f"Search: '{good_search.description}'")

print("\n💡 The difference:")
print("- Good descriptions explain WHEN to use the tool")
print("- They specify input/output formats")
print("- They include examples or use cases")
print("- They differentiate from similar tools")


In [ ]:
# From: tool_routing_patterns.py

# From: Zero to AI Agent, Chapter 12, Section 12.5
# File: tool_routing_patterns.py

from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

# Pattern 1: Hierarchical Tools (General → Specific)
def quick_answer(question: str) -> str:
    return f"Quick answer: {question[:50]}..."

def detailed_answer(question: str) -> str:
    return f"Detailed analysis: {question} [500 words]..."

def expert_answer(question: str) -> str:
    return f"Expert consultation: {question} [with citations]..."

hierarchy_tools = [
    Tool(
        name="quick_answer",
        func=quick_answer,
        description="For simple questions needing fast, brief answers (1-2 sentences)"
    ),
    Tool(
        name="detailed_answer",
        func=detailed_answer,
        description="For complex questions needing thorough explanation (paragraph+)"
    ),
    Tool(
        name="expert_answer",
        func=expert_answer,
        description="For technical questions needing citations and expertise"
    )
]

# Pattern 2: Domain-Specific Tools
def search_products(query: str) -> str:
    return f"Product results: {query}"

def search_documentation(query: str) -> str:
    return f"Documentation: {query}"

def search_forums(query: str) -> str:
    return f"Forum discussions: {query}"

domain_tools = [
    Tool(
        name="product_search",
        func=search_products,
        description="Search for products, prices, and shopping information"
    ),
    Tool(
        name="docs_search",
        func=search_documentation,
        description="Search technical documentation, APIs, and guides"
    ),
    Tool(
        name="forum_search",
        func=search_forums,
        description="Search community forums, discussions, and Q&A"
    )
]

# Test routing
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

print("ROUTING PATTERN EXAMPLES")
print("=" * 50)

# Test hierarchical routing
print("\n1. HIERARCHICAL ROUTING:")
test_hierarchical = [
    "What's 2+2?",  # Should use quick_answer
    "Explain how neural networks work",  # Should use detailed_answer
    "Provide peer-reviewed analysis of CRISPR technology"  # Should use expert_answer
]

llm_hierarchical = llm.bind_tools(hierarchy_tools)
for query in test_hierarchical:
    response = llm_hierarchical.invoke([HumanMessage(content=query)])
    if response.tool_calls:
        print(f"'{query[:30]}...' → {response.tool_calls[0]['name']}")

# Test domain routing
print("\n2. DOMAIN-SPECIFIC ROUTING:")
test_domains = [
    "Find the best laptop under $1000",  # Should use product_search
    "How to use pandas DataFrame",  # Should use docs_search
    "Why is my Python code slow?"  # Should use forum_search
]

llm_domain = llm.bind_tools(domain_tools)
for query in test_domains:
    response = llm_domain.invoke([HumanMessage(content=query)])
    if response.tool_calls:
        print(f"'{query[:30]}...' → {response.tool_calls[0]['name']}")


In [ ]:
# From: preventing_tool_loops.py

# From: Zero to AI Agent, Chapter 12, Section 12.5
# File: preventing_tool_loops.py

from langchain_classic.agents import AgentExecutor, create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langsmith import Client
from dotenv import load_dotenv
import time
import os

load_dotenv()

# Create a tool that might cause loops
call_count = 0

def problematic_search(query: str) -> str:
    global call_count
    call_count += 1
    # This tool returns questions instead of answers - loop risk!
    return f"Did you mean to search for '{query}'? Try being more specific."

def good_search(query: str) -> str:
    global call_count
    call_count += 1
    # This tool returns definitive answers - no loop risk
    return f"Found 5 results for '{query}': Result 1, Result 2, Result 3..."

# Create tools
problematic_tool = Tool(
    name="problematic_search",
    func=problematic_search,
    description="Search for information (might ask for clarification)"
)

good_tool = Tool(
    name="good_search",
    func=good_search,
    description="Search for information (returns definitive results)"
)

# Strategy 1: Set max_iterations
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=langsmith_api_key)
prompt = client.pull_prompt("hwchase17/react")

print("PREVENTING TOOL LOOPS")
print("=" * 50)

# Test with problematic tool (with safety limit)
print("\n1. WITH LOOP PREVENTION (max_iterations=3):")
call_count = 0
agent_safe = create_react_agent(llm, [problematic_tool], prompt)
executor_safe = AgentExecutor(
    agent=agent_safe,
    tools=[problematic_tool],
    max_iterations=3,  # Safety limit!
    verbose=True
)

try:
    result = executor_safe.invoke({"input": "Find information about Python"})
    print(f"Call count: {call_count}")
    print(f"Result: {result['output']}")
except Exception as e:
    print(f"Stopped after {call_count} iterations")

# Test with good tool (no loop risk)
print("\n2. WITH GOOD TOOL DESIGN (returns answers):")
call_count = 0
agent_good = create_react_agent(llm, [good_tool], prompt)
executor_good = AgentExecutor(
    agent=agent_good,
    tools=[good_tool],
    max_iterations=3,
    verbose=False
)

result = executor_good.invoke({"input": "Find information about Python"})
print(f"Call count: {call_count}")
print(f"Result: {result['output'][:100]}...")

print("\n💡 Loop Prevention Strategies:")
print("1. Always set max_iterations (3-5 is usually enough)")
print("2. Tools should return answers, not questions")
print("3. Tools should indicate when they're done")
print("4. Use early_stopping_method='generate' for natural stops")

In [ ]:
# From: multi_tool_orchestration.py

# From: Zero to AI Agent, Chapter 12, Section 12.5
# File: multi_tool_orchestration.py

from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_classic.agents import create_react_agent, AgentExecutor
from langsmith import Client
from dotenv import load_dotenv
from datetime import datetime
import os

load_dotenv()

# Create a suite of complementary tools
def get_date(dummy_input: str = "") -> str:
    """Get today's date. The dummy_input is ignored."""
    return datetime.now().strftime("%Y-%m-%d")

def search_events(input_str: str) -> str:
    """Search for events. Input format: 'date|location'"""
    try:
        date, location = input_str.split("|")
        return f"Events on {date} in {location}: Concert at 7pm, Festival at noon"
    except:
        return "Error: Please provide input as 'date|location'"

def check_weather(input_str: str) -> str:
    """Check weather. Input format: 'date|location'"""
    try:
        date, location = input_str.split("|")
        return f"Weather for {location} on {date}: Sunny, 75°F"
    except:
        return "Error: Please provide input as 'date|location'"

def make_recommendation(input_str: str) -> str:
    """Make recommendation. Input format: 'events|weather'"""
    try:
        events, weather = input_str.split("|")
        if "Sunny" in weather and "Festival" in events:
            return "Perfect day for the outdoor festival!"
        elif "Concert" in events:
            return "Evening concert would be great!"
        return "Consider indoor activities."
    except:
        return "Error: Please provide input as 'events|weather'"

# Design tools to work together
tools = [
    Tool(
        name="get_current_date",
        func=get_date,
        description="Get today's date in YYYY-MM-DD format. Just call without arguments."
    ),
    Tool(
        name="search_events",
        func=search_events,
        description="Find events. Input: 'date|location' like '2024-03-15|New York'"
    ),
    Tool(
        name="check_weather",
        func=check_weather,
        description="Check weather. Input: 'date|location' like '2024-03-15|New York'"
    ),
    Tool(
        name="make_recommendation",
        func=make_recommendation,
        description="Make recommendation. Input: 'events info|weather info'"
    )
]

# Initialize LLM and get prompt
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Get the ReAct prompt from LangSmith
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=langsmith_api_key)
prompt = client.pull_prompt("hwchase17/react")

# Create agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=5
)

# Test orchestration
test_queries = [
    "What should I do in New York today?",
    "Find events and weather for Boston tomorrow"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print('='*60)
    
    result = agent_executor.invoke({"input": query})
    print(f"\nFinal Answer: {result['output']}")

---
### Section 12.5 Exercises

### Exercise 12.5.1: Tool Naming Challenge (Easy)

You have 5 search-related functions. Create clear, distinctive names and descriptions for each:
- General web search
- News search
- Academic paper search
- Local business search
- Social media search

Make sure the agent would never confuse them!

In [ ]:
# Your code here


### Exercise 12.5.2: Loop Prevention (Medium)

Create a scenario with 3 tools where one might cause loops:
- A question-answering tool
- A clarification tool (potential loop risk!)
- A definition tool

Implement strategies to prevent the agent from getting stuck asking for clarification repeatedly. Test with various queries.

In [ ]:
# Your code here


### Exercise 12.5.3: Complex Orchestration (Hard)

Build a travel planning system with 6+ tools:
- Date/time tools
- Weather checking
- Flight searching
- Hotel searching
- Activity recommendations
- Itinerary creation

Create an agent that can handle: "Plan a 3-day trip to Tokyo next month"
The agent should orchestrate all tools to create a complete plan!

In [ ]:
# Your code here


---
## Section 12.6: Error handling in tool execution

In [ ]:
# From: basic_error_handling.py

# From: Zero to AI Agent, Chapter 12, Section 12.6
# File: basic_error_handling.py

from langchain_core.tools import Tool

# ❌ BAD: Tool that crashes
def bad_calculator(expression: str) -> str:
    result = eval(expression)  # This will crash on bad input!
    return str(result)

# ✅ GOOD: Tool that handles errors
def good_calculator(expression: str) -> str:
    try:
        result = eval(expression)
        return str(result)
    except ZeroDivisionError:
        return "Error: Cannot divide by zero"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: Calculation failed - {str(e)}"

# Test both versions
test_cases = [
    "10 + 5",      # Works fine
    "10 / 0",      # Division by zero
    "10 +",        # Syntax error
    "hello",       # Name error
]

print("COMPARING ERROR HANDLING")
print("=" * 50)

# Create tools
bad_tool = Tool(name="BadCalc", func=bad_calculator, description="Unsafe calculator")
good_tool = Tool(name="GoodCalc", func=good_calculator, description="Safe calculator")

for expression in test_cases:
    print(f"\nTesting: {expression}")
    
    # Try bad tool (wrapped to catch crashes)
    try:
        bad_result = bad_tool.func(expression)
        print(f"  Bad tool: {bad_result}")
    except Exception as e:
        print(f"  Bad tool: 💥 CRASHED - {e}")
    
    # Good tool always returns something
    good_result = good_tool.func(expression)
    print(f"  Good tool: {good_result}")


In [ ]:
# From: input_validation.py

# From: Zero to AI Agent, Chapter 12, Section 12.6
# File: input_validation.py

from langchain_core.tools import Tool
from datetime import datetime
import re

def search_with_validation(query: str) -> str:
    """
    Search tool with comprehensive input validation.
    """
    # 1. Check if input exists
    if not query:
        return "Error: Search query cannot be empty"
    
    # 2. Check type
    if not isinstance(query, str):
        return "Error: Search query must be text"
    
    # 3. Check length
    if len(query) < 2:
        return "Error: Search query too short (minimum 2 characters)"
    if len(query) > 200:
        return "Error: Search query too long (maximum 200 characters)"
    
    # 4. Clean dangerous characters
    # Remove special characters that might break the search
    cleaned = re.sub(r'[^\w\s\-.]', '', query)
    
    # 5. Check if anything remains
    if not cleaned.strip():
        return "Error: Search query contains only special characters"
    
    # If we get here, input is valid!
    return f"Searching for: '{cleaned}'"

def date_parser_with_validation(date_string: str) -> str:
    """
    Parse dates with validation and multiple format support.
    """
    if not date_string:
        return "Error: Date cannot be empty"
    
    # Try multiple date formats
    formats = [
        "%Y-%m-%d",      # 2024-03-15
        "%m/%d/%Y",      # 03/15/2024
        "%d/%m/%Y",      # 15/03/2024
        "%B %d, %Y",     # March 15, 2024
        "%b %d, %Y",     # Mar 15, 2024
    ]
    
    for fmt in formats:
        try:
            parsed_date = datetime.strptime(date_string.strip(), fmt)
            # Return in standard format
            return f"Date: {parsed_date.strftime('%Y-%m-%d')}"
        except ValueError:
            continue  # Try next format
    
    # If no format worked
    return f"Error: Could not parse date '{date_string}'. Try formats like: 2024-03-15, 03/15/2024, or March 15, 2024"

# Create validated tools
search_tool = Tool(
    name="ValidatedSearch",
    func=search_with_validation,
    description="Search with input validation"
)

date_tool = Tool(
    name="DateParser",
    func=date_parser_with_validation,
    description="Parse dates in various formats"
)

# Test validation
print("INPUT VALIDATION EXAMPLES")
print("=" * 50)

# Test search validation
search_tests = [
    "Python programming",  # Valid
    "",                    # Empty
    "a",                   # Too short
    "x" * 250,            # Too long
    "'; DROP TABLE;",     # SQL injection attempt
    "!!!***!!!",          # Only special chars
]

print("\nSearch Validation:")
for query in search_tests:
    display = query[:30] + "..." if len(query) > 30 else query
    result = search_tool.func(query)
    status = "✅" if not result.startswith("Error") else "❌"
    print(f"{status} '{display}' → {result}")

# Test date parsing
print("\nDate Parsing:")
date_tests = [
    "2024-03-15",
    "03/15/2024",
    "March 15, 2024",
    "invalid",
    "",
]

for date in date_tests:
    result = date_tool.func(date)
    status = "✅" if not result.startswith("Error") else "❌"
    print(f"{status} '{date}' → {result}")


In [ ]:
# From: external_service_handling.py

# From: Zero to AI Agent, Chapter 12, Section 12.6
# File: external_service_handling.py

from langchain_core.tools import Tool
import time
import random

# Simulate an unreliable external service
def unreliable_weather_api(city: str) -> dict:
    """Simulates an API that sometimes fails."""
    # 30% chance of failure
    if random.random() < 0.3:
        raise Exception("Service unavailable")
    # 20% chance of timeout
    if random.random() < 0.2:
        time.sleep(5)  # Simulate timeout
        raise TimeoutError("Request timed out")
    # Otherwise return data
    return {"city": city, "temp": 72, "conditions": "Sunny"}

def robust_weather_tool(city: str) -> str:
    """
    Weather tool with retry logic and fallbacks.
    """
    if not city:
        return "Error: City name required"
    
    # Try up to 3 times
    for attempt in range(3):
        try:
            # Set a timeout
            import signal
            
            def timeout_handler(signum, frame):
                raise TimeoutError("API call timed out")
            
            # Note: signal doesn't work on Windows, this is for illustration
            # In production, use requests library with timeout parameter
            
            # Make the API call
            result = unreliable_weather_api(city)
            
            # Success! Format and return
            return f"Weather in {result['city']}: {result['temp']}°F, {result['conditions']}"
            
        except TimeoutError:
            if attempt < 2:  # Don't sleep on last attempt
                print(f"  Attempt {attempt + 1} timed out, retrying...")
                time.sleep(1)  # Brief pause before retry
            continue
            
        except Exception as e:
            if attempt < 2:
                print(f"  Attempt {attempt + 1} failed: {e}, retrying...")
                time.sleep(1)
            continue
    
    # All attempts failed - return graceful error
    return f"Error: Unable to get weather for {city}. Please try again later."

# Alternative: Fallback to a different service
def weather_with_fallback(city: str) -> str:
    """
    Try primary service, fallback to secondary if needed.
    """
    # Try primary service
    try:
        # Primary API call (simulated)
        if random.random() < 0.5:  # 50% failure rate
            raise Exception("Primary API failed")
        return f"Weather from PRIMARY: {city} is 72°F"
    except:
        # Fallback to secondary service
        try:
            # Secondary API call (simulated)
            if random.random() < 0.3:  # 30% failure rate
                raise Exception("Secondary API failed")
            return f"Weather from BACKUP: {city} is 70°F"
        except:
            # Both failed
            return f"Error: Weather services unavailable for {city}"

# Test the robust tool
print("TESTING EXTERNAL SERVICE HANDLING")
print("=" * 50)

tool = Tool(
    name="RobustWeather",
    func=robust_weather_tool,
    description="Get weather with retry logic"
)

print("\nTesting with retries (watch for retry messages):")
for i in range(5):
    result = tool.func("New York")
    print(f"Attempt {i+1}: {result}")
    print()


In [ ]:
# From: graceful_degradation.py

# From: Zero to AI Agent, Chapter 12, Section 12.6
# File: graceful_degradation.py

from langchain_core.tools import Tool
from datetime import datetime
import json
import random

def smart_search_tool(query: str, options: str = "{}") -> str:
    """
    Search tool that degrades gracefully based on available services.
    """
    # Parse options
    try:
        opts = json.loads(options)
    except:
        opts = {}
    
    results = []
    
    # Try premium search (most detailed)
    try:
        if "premium" in opts and opts["premium"]:
            # Simulate premium API
            premium_result = f"PREMIUM: Detailed analysis of '{query}' with citations"
            results.append(premium_result)
    except:
        pass  # Premium failed, continue with others
    
    # Try standard search
    try:
        # Simulate standard search
        if random.random() > 0.3:  # 70% success rate
            standard_result = f"STANDARD: Basic information about '{query}'"
            results.append(standard_result)
    except:
        pass  # Standard failed, continue
    
    # Fallback: Use cached results
    try:
        # Simulate cache lookup
        cache = {
            "python": "Python is a programming language",
            "weather": "Weather varies by location",
            "news": "Latest updates from various sources"
        }
        
        for key in cache:
            if key.lower() in query.lower():
                results.append(f"CACHED: {cache[key]}")
                break
    except:
        pass
    
    # Ultimate fallback
    if not results:
        # Provide generic but helpful response
        return f"Unable to search for '{query}' at this time. Try: 1) Simplifying your query, 2) Checking your connection, 3) Trying again in a moment"
    
    # Return best available results
    return " | ".join(results)

def calculation_with_fallback(expression: str) -> str:
    """
    Calculator that falls back to simpler operations if complex ones fail.
    """
    # Try advanced calculation
    try:
        # Attempt with math module for complex operations
        import math
        # Create safe namespace with math functions
        safe_dict = {
            'sin': math.sin, 'cos': math.cos, 'tan': math.tan,
            'sqrt': math.sqrt, 'log': math.log, 'pi': math.pi,
            'e': math.e
        }
        result = eval(expression, {"__builtins__": {}}, safe_dict)
        return f"Result: {result}"
    except:
        pass
    
    # Fallback to basic arithmetic
    try:
        # Only allow basic operations
        if all(c in '0123456789+-*/() .' for c in expression):
            result = eval(expression)
            return f"Result (basic): {result}"
    except:
        pass
    
    # Ultimate fallback: Explain what went wrong
    return f"Error: Cannot calculate '{expression}'. Try using only numbers and basic operations (+, -, *, /)."

print("GRACEFUL DEGRADATION EXAMPLES")
print("=" * 50)

# Test search degradation
search = Tool(name="SmartSearch", func=smart_search_tool, description="Search with fallbacks")

queries = ["python", "quantum computing", "xyz123abc"]
for q in queries:
    print(f"\nSearching: {q}")
    print(f"Result: {search.func(q)}")

# Test calculation degradation  
calc = Tool(name="SmartCalc", func=calculation_with_fallback, description="Calculate with fallbacks")

expressions = ["2+2", "sin(3.14/2)", "invalid expression"]
for expr in expressions:
    print(f"\nCalculating: {expr}")
    print(f"Result: {calc.func(expr)}")


In [ ]:
# From: helpful_error_messages.py

# From: Zero to AI Agent, Chapter 12, Section 12.6
# File: helpful_error_messages.py

from langchain_core.tools import Tool

def file_tool_with_helpful_errors(command: str) -> str:
    """
    File tool that provides actionable error messages.
    """
    parts = command.split()
    if not parts:
        return """Error: No command provided.
        
How to use this tool:
- 'read <filename>' - Read a file
- 'write <filename> <content>' - Write to a file
- 'list' - List files in current directory
        
Example: 'read document.txt'"""
    
    action = parts[0].lower()
    
    if action == "read":
        if len(parts) < 2:
            return """Error: Filename required for read command.
            
Correct format: 'read <filename>'
Example: 'read report.pdf'
            
Available files: document.txt, data.csv, notes.md"""
        
        filename = parts[1]
        
        # Simulate file not found
        if filename not in ["document.txt", "data.csv", "notes.md"]:
            return f"""Error: File '{filename}' not found.
            
Did you mean one of these?
- document.txt
- data.csv  
- notes.md
            
To see all files, use: 'list'"""
        
        return f"Contents of {filename}: [file contents here]"
    
    elif action == "write":
        if len(parts) < 3:
            return """Error: Write command requires filename and content.
            
Correct format: 'write <filename> <content>'
Example: 'write notes.txt Meeting at 3pm'
            
Note: Content will be written as a single line."""
        
        return f"Successfully wrote to {parts[1]}"
    
    elif action == "list":
        return """Files in current directory:
- document.txt (1.2 KB) - Last modified: 2024-03-15
- data.csv (5.6 KB) - Last modified: 2024-03-14  
- notes.md (850 B) - Last modified: 2024-03-15"""
    
    else:
        return f"""Error: Unknown command '{action}'.
        
Available commands:
- read: Read a file
- write: Write to a file
- list: Show all files
        
Type the command followed by any required parameters.
Example: 'read document.txt'"""

# Test helpful errors
print("HELPFUL ERROR MESSAGES")
print("=" * 50)

tool = Tool(
    name="FileTool",
    func=file_tool_with_helpful_errors,
    description="File operations with helpful errors"
)

test_commands = [
    "",                    # No command
    "read",               # Missing filename
    "read missing.txt",   # File not found
    "write notes.txt",    # Missing content
    "invalid",            # Unknown command
    "read document.txt",  # Success case
]

for cmd in test_commands:
    print(f"\nCommand: '{cmd}'")
    print("-" * 40)
    result = tool.func(cmd)
    print(result)


In [ ]:
# From: tool_logging.py

# From: Zero to AI Agent, Chapter 12, Section 12.6
# File: tool_logging.py

from langchain_core.tools import Tool
import logging
from datetime import datetime
import json

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

class ToolMonitor:
    """Simple tool monitoring system."""
    
    def __init__(self):
        self.stats = {
            "total_calls": 0,
            "successful_calls": 0,
            "failed_calls": 0,
            "errors": []
        }
    
    def log_call(self, tool_name: str, input_data: str, result: str):
        """Log a tool call."""
        self.stats["total_calls"] += 1
        
        if result.startswith("Error"):
            self.stats["failed_calls"] += 1
            self.stats["errors"].append({
                "tool": tool_name,
                "input": input_data,
                "error": result,
                "time": datetime.now().isoformat()
            })
            logging.error(f"Tool {tool_name} failed: {result}")
        else:
            self.stats["successful_calls"] += 1
            logging.info(f"Tool {tool_name} succeeded")
    
    def get_report(self):
        """Get monitoring report."""
        success_rate = (
            self.stats["successful_calls"] / self.stats["total_calls"] * 100
            if self.stats["total_calls"] > 0 else 0
        )
        
        return f"""
Tool Monitoring Report
=====================
Total Calls: {self.stats["total_calls"]}
Successful: {self.stats["successful_calls"]}
Failed: {self.stats["failed_calls"]}
Success Rate: {success_rate:.1f}%

Recent Errors: {len(self.stats["errors"])}
"""

# Create a monitored tool
monitor = ToolMonitor()

def monitored_calculator(expression: str) -> str:
    """Calculator with monitoring."""
    tool_name = "Calculator"
    
    try:
        result = str(eval(expression))
        monitor.log_call(tool_name, expression, result)
        return result
    except Exception as e:
        error_msg = f"Error: {str(e)}"
        monitor.log_call(tool_name, expression, error_msg)
        return error_msg

# Test monitoring
print("TOOL MONITORING EXAMPLE")
print("=" * 50)

tool = Tool(
    name="MonitoredCalc",
    func=monitored_calculator,
    description="Calculator with monitoring"
)

# Run various calculations
test_cases = [
    "10 + 5",     # Success
    "20 * 3",     # Success  
    "100 / 0",    # Error
    "50 - 30",    # Success
    "invalid",    # Error
]

for expr in test_cases:
    result = tool.func(expr)
    print(f"{expr} = {result}")

# Show monitoring report
print("\n" + monitor.get_report())


---
### Section 12.6 Exercises

### Exercise 12.6.1: Robust URL Fetcher (Easy)

Create a tool that fetches webpage content with:
- URL validation (must start with http:// or https://)
- Timeout handling (max 5 seconds)
- Retry logic (up to 2 retries)
- Helpful error messages for common issues

Test with valid URLs, invalid URLs, and slow/failing endpoints.

In [ ]:
# Your code here


### Exercise 12.6.2: Smart Data Processor (Medium)

Build a tool that processes CSV data with multiple fallback strategies:
- Try to parse as CSV
- If that fails, try TSV (tab-separated)
- If that fails, try space-separated
- Return helpful error with sample of expected format

Include validation for minimum/maximum rows and columns.

In [ ]:
# Your code here


### Exercise 12.6.3: Resilient API Client (Hard)

Create a tool that calls an external API with:
- Rate limiting (max 10 calls per minute)
- Exponential backoff on failures
- Circuit breaker pattern (stop trying after 5 consecutive failures)
- Cache successful responses for 5 minutes
- Detailed logging of all attempts

Simulate various failure scenarios and verify your tool handles them all.

In [ ]:
# Your code here


---
## Section 12.7: Building a multi-tool agent

In [ ]:
# From: first_agent.py

# From: Zero to AI Agent, Chapter 12, Section 12.7
# File: first_agent.py

from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_classic.agents import create_react_agent, AgentExecutor
from langsmith import Client
from dotenv import load_dotenv
import os

load_dotenv()

# Step 1: Create a tool
def multiply(numbers: str) -> str:
    """Multiply two numbers separated by comma."""
    try:
        a, b = map(float, numbers.split(','))
        return str(a * b)
    except:
        return "Error: Please provide two numbers separated by comma"

multiply_tool = Tool(
    name="Multiplier",
    func=multiply,
    description="Multiply two numbers. Input format: 'number1,number2'"
)

# Step 2: Create the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Step 3: Get the ReAct prompt from LangSmith
# You need LANGSMITH_API_KEY from https://smith.langchain.com/
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=langsmith_api_key)
prompt = client.pull_prompt("hwchase17/react")

# Step 4: Create the agent
agent = create_react_agent(
    llm=llm,
    tools=[multiply_tool],
    prompt=prompt
)

# Step 5: Create the executor (this runs the agent)
agent_executor = AgentExecutor(
    agent=agent,
    tools=[multiply_tool],
    verbose=True,  # See the thinking process!
    max_iterations=3  # Safety limit
)

# Step 6: Use your agent!
result = agent_executor.invoke({
    "input": "What is 15 times 24?"
})

print(f"\nFinal Answer: {result['output']}")


In [ ]:
# From: agent_without_hub.py

# From: Zero to AI Agent, Chapter 12, Section 12.7
# File: agent_without_hub.py

from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_classic.agents import AgentExecutor
from langchain_core.prompts import PromptTemplate
from langchain_classic.agents.format_scratchpad import format_log_to_str
from langchain_classic.agents.output_parsers import ReActSingleInputOutputParser
from dotenv import load_dotenv

load_dotenv()

# Create a tool
def multiply(numbers: str) -> str:
    """Multiply two numbers separated by comma."""
    try:
        a, b = map(float, numbers.split(','))
        return str(a * b)
    except:
        return "Error: Please provide two numbers separated by comma"

multiply_tool = Tool(
    name="Multiplier",
    func=multiply,
    description="Multiply two numbers. Input format: 'number1,number2'"
)

# Create the ReAct prompt manually (this is what hub.pull does)
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

# Create LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Bind tools to LLM
llm_with_stop = llm.bind(stop=["\nObservation"])

# Create the agent chain
agent = (
    {
        "input": lambda x: x["input"],
        "tools": lambda x: "\n".join([f"{tool.name}: {tool.description}" for tool in [multiply_tool]]),
        "tool_names": lambda x: ", ".join([tool.name for tool in [multiply_tool]]),
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

# Create executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[multiply_tool],
    verbose=True
)

# Use it!
result = agent_executor.invoke({"input": "What is 15 times 24?"})
print(f"\nFinal Answer: {result['output']}")


In [ ]:
# From: multi_tool_agent.py

# From: Zero to AI Agent, Chapter 12, Section 12.7
# File: multi_tool_agent.py

from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_classic.agents import create_react_agent, AgentExecutor
from langsmith import Client
from dotenv import load_dotenv
from datetime import datetime
import random
import os

load_dotenv()

# Create diverse tools
def calculate(expression: str) -> str:
    """Perform mathematical calculations."""
    try:
        # Safety: only allow numbers and basic operations
        allowed = set('0123456789+-*/().')
        if all(c in allowed or c.isspace() for c in expression):
            result = eval(expression)
            return str(result)
        return "Error: Only basic math operations allowed"
    except Exception as e:
        return f"Error: {str(e)}"

def get_time(location: str = "local") -> str:
    """Get current time."""
    current_time = datetime.now()
    return f"Current time: {current_time.strftime('%I:%M %p, %A, %B %d, %Y')}"

def get_weather(city: str) -> str:
    """Get weather for a city (simulated)."""
    # In production, this would call a real weather API
    weather_data = {
        "New York": {"temp": 72, "condition": "Partly cloudy"},
        "London": {"temp": 59, "condition": "Rainy"},
        "Tokyo": {"temp": 68, "condition": "Clear"},
        "Paris": {"temp": 64, "condition": "Cloudy"},
    }
    
    if city in weather_data:
        data = weather_data[city]
        return f"Weather in {city}: {data['temp']}°F, {data['condition']}"
    else:
        return f"Weather data not available for {city}"

def search_info(query: str) -> str:
    """Search for information (simulated)."""
    # In production, this would use a real search API
    responses = {
        "python": "Python is a high-level programming language known for its simplicity.",
        "langchain": "LangChain is a framework for building applications with LLMs.",
        "agent": "An AI agent is a system that can perceive, reason, and act autonomously.",
    }
    
    query_lower = query.lower()
    for key, value in responses.items():
        if key in query_lower:
            return value
    
    return f"Information about '{query}' is not in my current database."

# Create tool objects
tools = [
    Tool(
        name="Calculator",
        func=calculate,
        description="Perform mathematical calculations. Input: math expression like '2+2' or '15*3.14'"
    ),
    Tool(
        name="Clock",
        func=get_time,
        description="Get the current date and time. Input: location (or 'local' for local time)"
    ),
    Tool(
        name="Weather",
        func=get_weather,
        description="Get current weather for a city. Input: city name"
    ),
    Tool(
        name="Search",
        func=search_info,
        description="Search for information about a topic. Input: search query"
    ),
]

# Create the agent
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=langsmith_api_key)
prompt = client.pull_prompt("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)

# Create executor with safety limits
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=5,  # Prevent infinite loops
    handle_parsing_errors=True  # Handle any parsing issues
)

print("🤖 MULTI-TOOL AI AGENT READY!")
print("=" * 50)

# Test with various queries
test_queries = [
    "What's 25 * 4?",
    "What time is it?",
    "What's the weather in London?",
    "Tell me about Python",
    "What's the weather in Paris and what's 100 divided by 4?",  # Multiple tools!
]

for query in test_queries:
    print(f"\n📝 User: {query}")
    print("-" * 40)
    
    result = agent_executor.invoke({"input": query})
    
    print(f"\n🤖 Agent: {result['output']}")
    print("=" * 50)


In [ ]:
# From: research_assistant.py

# From: Zero to AI Agent, Chapter 12, Section 12.7
# File: research_assistant.py

from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import Tool
from langchain_classic.agents import create_react_agent, AgentExecutor
from langsmith import Client
from dotenv import load_dotenv
import json
import os

load_dotenv()

# Setup built-in tools
search_tool = DuckDuckGoSearchRun()
wikipedia_tool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(
        top_k_results=1,
        doc_content_chars_max=500
    )
)

# Create custom tools for the assistant
def save_notes(content: str) -> str:
    """Save research notes to a file."""
    try:
        # In production, implement proper file handling
        with open("research_notes.txt", "a") as f:
            f.write(f"\n{content}\n")
        return "Notes saved successfully"
    except Exception as e:
        return f"Error saving notes: {e}"

def summarize(text: str) -> str:
    """Create a brief summary of text."""
    # Simple summary (in production, might use another LLM call)
    words = text.split()
    if len(words) > 50:
        summary = ' '.join(words[:50]) + "..."
    else:
        summary = text
    return f"Summary: {summary}"

# Combine all tools
tools = [
    search_tool,
    wikipedia_tool,
    Tool(name="SaveNotes", func=save_notes, 
         description="Save important information to notes. Input: text to save"),
    Tool(name="Summarize", func=summarize,
         description="Create a brief summary. Input: text to summarize"),
]

# Create research assistant
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=langsmith_api_key)
prompt = client.pull_prompt("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=6
)

print("🔬 RESEARCH ASSISTANT READY!")
print("=" * 50)

# Complex research task
research_query = """
Research LangChain framework:
1. Find current information about it
2. Check Wikipedia for background
3. Save the key points to notes
4. Give me a summary
"""

print(f"📚 Research Request: {research_query}")
print("-" * 50)

result = agent_executor.invoke({"input": research_query})

print(f"\n📊 Research Complete!")
print(f"Final Report: {result['output']}")


In [ ]:
# From: personal_assistant_solution.py

# From: Zero to AI Agent, Chapter 12, Section 12.7
# File: personal_assistant_solution.py

from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_classic.agents import create_react_agent, AgentExecutor
from langsmith import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta
import json
import os

load_dotenv()

# 1. Time Management Tools
def get_time_zone(location: str) -> str:
    """Get time in different timezones."""
    timezone_offsets = {
        "NYC": -5, "London": 0, "Tokyo": 9, 
        "Paris": 1, "Sydney": 11
    }
    
    if location in timezone_offsets:
        utc_now = datetime.utcnow()
        local_time = utc_now + timedelta(hours=timezone_offsets[location])
        return f"Time in {location}: {local_time.strftime('%I:%M %p')}"
    return f"Unknown timezone for {location}"

def set_reminder(reminder: str) -> str:
    """Save a reminder to file."""
    try:
        with open("reminders.txt", "a") as f:
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M')
            f.write(f"[{timestamp}] {reminder}\n")
        return f"Reminder set: {reminder}"
    except:
        return "Error setting reminder"

# 2. Task Management Tools
def manage_todo(action_and_task: str) -> str:
    """Manage todo list. Input: 'add:task' or 'complete:task' or 'list'"""
    parts = action_and_task.split(':')
    action = parts[0].lower()
    
    todos_file = "todos.json"
    
    # Load existing todos
    if os.path.exists(todos_file):
        with open(todos_file, 'r') as f:
            todos = json.load(f)
    else:
        todos = []
    
    if action == "add" and len(parts) > 1:
        task = ':'.join(parts[1:])
        todos.append({"task": task, "done": False})
        with open(todos_file, 'w') as f:
            json.dump(todos, f)
        return f"Added task: {task}"
    
    elif action == "complete" and len(parts) > 1:
        task = ':'.join(parts[1:])
        for todo in todos:
            if todo["task"] == task:
                todo["done"] = True
        with open(todos_file, 'w') as f:
            json.dump(todos, f)
        return f"Completed: {task}"
    
    elif action == "list":
        pending = [t["task"] for t in todos if not t["done"]]
        return f"Pending tasks: {', '.join(pending) if pending else 'None'}"
    
    return "Usage: 'add:task', 'complete:task', or 'list'"

# 3. Calculation Tools
def unit_converter(conversion_request: str) -> str:
    """Convert units. Input: 'value unit to unit' like '10 meters to feet'"""
    conversions = {
        ("meters", "feet"): 3.28084,
        ("feet", "meters"): 0.3048,
        ("kg", "pounds"): 2.20462,
        ("celsius", "fahrenheit"): lambda c: c * 9/5 + 32,
    }
    
    try:
        parts = conversion_request.lower().split()
        value = float(parts[0])
        from_unit = parts[1]
        to_unit = parts[3]
        
        key = (from_unit, to_unit)
        if key in conversions:
            if callable(conversions[key]):
                result = conversions[key](value)
            else:
                result = value * conversions[key]
            return f"{value} {from_unit} = {result:.2f} {to_unit}"
    except:
        pass
    
    return "Conversion not available. Try: '10 meters to feet'"

# 4. Weather Tool (enhanced)
def weather_assistant(city: str) -> str:
    """Get weather and clothing suggestions."""
    weather_data = {
        "London": {"temp": 59, "condition": "Rainy"},
        "Paris": {"temp": 64, "condition": "Cloudy"},
        "NYC": {"temp": 72, "condition": "Sunny"},
    }
    
    if city in weather_data:
        data = weather_data[city]
        suggestion = ""
        
        if "rain" in data["condition"].lower():
            suggestion = " Bring an umbrella!"
        elif data["temp"] < 60:
            suggestion = " Wear a jacket!"
        elif data["temp"] > 75:
            suggestion = " Light clothing recommended!"
        
        return f"Weather in {city}: {data['temp']}°F, {data['condition']}.{suggestion}"
    
    return f"No weather data for {city}"

# Create all tools
tools = [
    Tool(name="TimeZone", func=get_time_zone, 
         description="Get time in a city. Input: city name like 'NYC' or 'London'"),
    Tool(name="Reminder", func=set_reminder,
         description="Set a reminder. Input: reminder text"),
    Tool(name="TodoList", func=manage_todo,
         description="Manage todos. Input: 'add:task', 'complete:task', or 'list'"),
    Tool(name="UnitConverter", func=unit_converter,
         description="Convert units. Input: 'value from_unit to to_unit'"),
    Tool(name="Weather", func=weather_assistant,
         description="Get weather and clothing suggestions. Input: city name"),
]

# Create the personal assistant
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=langsmith_api_key)
prompt = client.pull_prompt("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=6,
    handle_parsing_errors=True
)

print("🤖 PERSONAL ASSISTANT READY!")
print("=" * 50)

# Test complex query from the challenge
test_query = """What's the weather in Paris and NYC, which is warmer, 
and add 'pack umbrella' to my todo list if either is rainy"""

print(f"User: {test_query}")
print("-" * 50)

result = agent_executor.invoke({"input": test_query})
print(f"\nAssistant: {result['output']}")


---
## Next Steps

- Check your answers in **chapter_12_tools_functions_solutions.ipynb**
- Proceed to **Chapter 13**